# Exercise - Text Mining - Classification - SCIKIT-LEARN

We will predict the category of a product based on the description of the product.

**The unit of analysis is a product**

In [1]:
import pandas as pd
import numpy as np

In [2]:
products = pd.read_csv('products.csv')

In [3]:
products.head(5)

,Category,Description
0,Electronics,HP 680 Original Ink Advantage Cartridge (Black...
1,Clothing & Accessories,Bold N Elegant Navy Blue Thin Summer Pregnancy...
2,Books,The Travel Book: A Journey Through Every Count...
3,Electronics,Tiny Deal Compact 10x25 Mini Binoculars Telesc...
4,Clothing & Accessories,Nimble House 16Pcs/Set Unisex Women Men No Tie...


In [4]:
products.shape

(5000, 2)

## Assign the "target" variable



In [11]:
target = products['Category']

## Assign the "text" (input) variable

In [12]:
input_data = products[['Description']]

## Split the data

In [13]:
from sklearn.model_selection import train_test_split

train_set, test_set, train_y, test_y = train_test_split(input_data, target, test_size=0.3, random_state=42)

In [8]:
train_set.shape, train_y.shape

((3500, 1), (3500,))

In [14]:
test_set.shape, test_y.shape

((1500, 1), (1500,))

# Data Prep

## Sklearn: Text preparation

Create a pipeline to transform the text column and create a term by document matrix.

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import FunctionTransformer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [17]:
def new_col(df):
    #Create a copy so that we don't overwrite the existing dataframe
    df1 = df.copy()
    
    # First, conver the dataframe column to a numpy array. Then, call the ravel function to make it one-dimensional
    return np.array(df).ravel()

In [18]:
new_col(train_set)

array(['Dehman 100-Percent Silky Satin Hair Beauty Pillowcase, Black, King Size DEHMANSATIN Silk originated in INDIA and it is an honor to present this ancient oriental gift to you. The collective wisdom of the sericulture people of this land has been condensed DEHMAN has always adhered to its original heart from generation to generation. From raw satin silk material collection to processing and to finished products,each step is carefully selected. DEHMAN hopes that every customer will have a wonderful satin silk experience and enjoy the silky noble life. Why Choose DEHMAN Pillowcase ? Material：DEHMAN Satin Silk pillowcase is crafted by 19 momme pure mulberry silk. Both sides of the pillowcase is organic and natural satin silk.Light weight and easy to carry. Design: It is designed with hidden zipper closure.With Queen(20x30inches) and King(20x36inches) and Standard size(20x26inches) in various kinds colours. Quality：Superior durable plain color ,not easy to run after washing.Exquisite 

In [20]:
text_column = ['Description']

In [44]:
number_svd_components = 500

In [45]:
text_transformer = Pipeline(steps=[
                ('my_new_column', FunctionTransformer(new_col)),
                ('text', TfidfVectorizer(stop_words='english')),
                ('svd', TruncatedSVD(n_components=number_svd_components, n_iter=10))
            ])

In [46]:
preprocessor = ColumnTransformer([
                     ('text', text_transformer, text_column),
                    ],
        remainder='drop')

#passtrough is an optional step. 

In [47]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_set)

train_x

array([[ 1.32127476e-01, -3.83815352e-02, -5.52127985e-02, ...,
         2.19671309e-02, -1.95717275e-02, -1.90545645e-02],
       [ 1.19672838e-03,  1.09071177e-03, -9.41521625e-06, ...,
        -3.07134638e-03,  8.67785154e-03,  6.44866776e-03],
       [ 5.74701078e-02,  5.16487292e-02,  8.81057288e-03, ...,
         1.61169437e-02, -7.52522684e-03, -1.14667795e-03],
       ...,
       [ 1.59120292e-01, -7.66943792e-02, -1.60781830e-01, ...,
        -2.52529891e-03, -1.63111933e-02,  8.17335898e-03],
       [ 8.31440381e-02, -4.33959857e-02, -3.44149656e-02, ...,
        -7.17453246e-03, -6.18195590e-03, -3.02089228e-02],
       [ 9.66538621e-02, -3.95970843e-02, -1.19857668e-01, ...,
         4.28482353e-02, -2.47389018e-02, -1.78786143e-02]])

In [48]:
# Transform the test data
test_x = preprocessor.transform(test_set)

test_x

array([[ 0.0197115 ,  0.0121762 ,  0.00564652, ...,  0.01736046,
         0.02325228, -0.00598611],
       [ 0.08027585, -0.04060881,  0.02606995, ...,  0.01262605,
         0.04518485,  0.02717815],
       [ 0.17327444, -0.07025064, -0.0460695 , ...,  0.00624072,
        -0.00765925, -0.01024442],
       ...,
       [ 0.08834409,  0.03663293,  0.00627167, ..., -0.00516674,
        -0.00452773, -0.01228504],
       [ 0.08826245, -0.03472008,  0.03563941, ...,  0.01446637,
         0.01764299, -0.01875452],
       [ 0.01982621, -0.00944311,  0.02909595, ..., -0.01503946,
        -0.00123765,  0.01915129]])

In [49]:
test_x.shape

(1500, 500)

# Calculate the baseline

In [50]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_y)

DummyClassifier(strategy='most_frequent')

In [51]:
from sklearn.metrics import accuracy_score

In [52]:
#Baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_y, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.38142857142857145


In [53]:
#Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_y, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.386


# Try one of the classifiers we have covered so far

In [54]:
from sklearn.ensemble import RandomForestClassifier 

from sklearn.metrics import accuracy_score

In [55]:
rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=16, n_jobs=-1) 

rnd_clf.fit(train_x, train_y)



RandomForestClassifier(max_leaf_nodes=16, n_jobs=-1)

## Accuracy

In [56]:
from sklearn.metrics import accuracy_score

In [57]:
#Train accuracy

train_y_pred = rnd_clf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

Train acc: 0.8848571428571429


In [58]:
#Test accuracy

test_y_pred = rnd_clf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

Test acc: 0.87


## Generate the confusion matrix

In [59]:
from sklearn.metrics import confusion_matrix

#Usually created on test set
confusion_matrix(test_y, test_y_pred)

array([[325,   5,   3,  32],
       [  1, 214,   2,  41],
       [ 10,   0, 204,  84],
       [ 12,   3,   2, 562]], dtype=int64)

# Try another classifier we have covered so far

In [60]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=100)


In [61]:
sgd_clf.fit(train_x, train_y)

SGDClassifier(max_iter=100)

## Accuracy

In [62]:
#Train accuracy

train_y_pred = sgd_clf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

Train acc: 0.9574285714285714


In [64]:
#Test accuracy

test_y_pred = sgd_clf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

Test acc: 0.9426666666666667


In [65]:
test_y_pred

array(['Books', 'Household', 'Household', ..., 'Books', 'Household',
       'Electronics'], dtype='<U22')

## Generate the confusion matrix

In [66]:
from sklearn.metrics import confusion_matrix

#Usually created on test set
confusion_matrix(test_y, test_y_pred)

array([[336,   6,   4,  19],
       [  1, 253,   1,   3],
       [  8,   1, 274,  15],
       [ 12,   7,   9, 551]], dtype=int64)

# Check for the number of components

**Determine whether increasing/decreasing the number of components increases/decreases the two models' accuracies** Discussing of the findings below.

The model is run with two different component values:

The test accuracy 
Baseline = 0.386

Components are set at = 300
Random Forest Classifer Model is 0.8993
The SGDClassifier model is 0.9366666666666666  
 

Comparison: with components = 500
Baseline = 0.386 (unchanged)
Random Forest Classifier = 0.87 (reduced) 
SGD Classifier = 0.942 (increased)